In [142]:
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime, date
import holidays

In [143]:
# load zonal demand data 
demand_files = glob.glob('../HourlyZonalDemand/PUB_DemandZonal_*.csv')
dfs = []

for file in demand_files:
    df = pd.read_csv(file, skiprows=3)  # Skip header rows
    dfs.append(df)

demand_df = pd.concat(dfs, ignore_index=True)

# Convert date and hour to datetime
demand_df['datetime'] = pd.to_datetime(demand_df['Date']) + pd.to_timedelta(demand_df['Hour'] - 1, unit='h')
demand_df = demand_df.sort_values('datetime')

# Filter demand_df to only include data until 2025-04-04
demand_df = demand_df[demand_df['datetime'] < '2025-04-05']

# drop unrequired columns
demand_df = demand_df.drop(columns=['Diff', 'Zone Total', "Date", "Hour"]) # we add hour back in again later for consistency 





In [144]:
display(demand_df)

,Ontario Demand,Northwest,Northeast,Ottawa,East,Toronto,Essa,Bruce,Southwest,Niagara,West,datetime
0,16627,591,1466,1066,1268,5340,1579,89,3157,477,1489,2018-01-01 00:00:00
1,16084,577,1420,985,1300,5211,1516,91,3061,462,1465,2018-01-01 01:00:00
2,15866,613,1417,943,1316,5096,1471,86,3032,446,1441,2018-01-01 02:00:00
3,15725,656,1430,943,1303,4987,1451,81,2911,439,1413,2018-01-01 03:00:00
4,15470,657,1425,932,1343,4926,1422,69,2907,449,1391,2018-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
63619,16611,641,1360,1007,1122,6004,1150,119,3139,565,1754,2025-04-04 19:00:00
63620,16209,633,1381,991,1058,5859,1120,113,2995,553,1698,2025-04-04 20:00:00
63621,15299,624,1363,954,1009,5537,1052,111,2842,521,1556,2025-04-04 21:00:00
63622,14235,613,1316,876,945,5198,972,108,2665,511,1447,2025-04-04 22:00:00


In [145]:
demand_df.tail(1000)

,Ontario Demand,Northwest,Northeast,Ottawa,East,Toronto,Essa,Bruce,Southwest,Niagara,West,datetime
62624,17786,611,1503,1153,1170,5949,1316,196,3229,605,2169,2025-02-22 08:00:00
62625,17969,617,1520,1181,1184,6171,1328,194,3312,609,2019,2025-02-22 09:00:00
62626,17885,615,1512,1209,1204,6312,1321,193,3342,615,1783,2025-02-22 10:00:00
62627,17935,615,1471,1230,1204,6435,1304,192,3377,607,1690,2025-02-22 11:00:00
62628,17743,591,1460,1230,1160,6342,1275,192,3359,591,1641,2025-02-22 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
63619,16611,641,1360,1007,1122,6004,1150,119,3139,565,1754,2025-04-04 19:00:00
63620,16209,633,1381,991,1058,5859,1120,113,2995,553,1698,2025-04-04 20:00:00
63621,15299,624,1363,954,1009,5537,1052,111,2842,521,1556,2025-04-04 21:00:00
63622,14235,613,1316,876,945,5198,972,108,2665,511,1447,2025-04-04 22:00:00


In [146]:
demand_df.shape

(63624, 12)

In [147]:
# load hourly climate data from different regions into a multi-level dataframe
climate_files = glob.glob('../ClimateData/*Climate.csv')
dfs = []

for file in climate_files:
    # Extract region name from filename (e.g., "NortheastClimate.csv" -> "Northeast")
    region = os.path.basename(file).split('Climate')[0]
    df = pd.read_csv(file, skiprows=3, nrows=63600) 
    # there are a bunch of missing values after 63604, i.e. April 5th data is incomplete so we shouldn't use it
    # data goes until line 63676 after that we have mean data for each day
    df['datetime'] = pd.to_datetime(df['time'])
    df['region'] = region
    df = df.drop(columns=["time"])
    
    # Set multi-index with datetime and region
    df.set_index(['datetime', 'region'], inplace=True)
    dfs.append(df)

# Combine all climate dataframes
climate_df = pd.concat(dfs)

# Sort the multi-index
climate_df.sort_index(inplace=True)

climate_df.head(1000)

temperature_2m (°C)  cloud_cover (%)  \
datetime            region                                            
2018-01-02 00:00:00 Bruce                     -6.8               98   
                    East                     -17.6               25   
                    Essa                     -22.6                4   
                    Niagara                  -12.4              100   
                    Northeast                -24.2              100   
...                                            ...              ...   
2018-01-06 03:00:00 Northwest                -31.1                0   
                    Ottawa                   -23.3               99   
                    Southwest                -22.3                1   
                    Toronto                  -22.8               99   
                    West                     -18.6                0   

                               cloud_cover_low (%)  cloud_cover_mid (%)  \
datetime            region                                                
2018-01-02 00:00:00 Bruce                       96                   98   
                    East                        25                    0   
                    Essa                         4                    0   
                    Niagara                    100                  100   
                    Northeast                    0                  100   
...                                            ...                  ...   
2018-01-06 03:00:00 Northwest                    0                    0   
                    Ottawa                      96                   95   
                    Southwest                    0                    1   
                    Toronto                     99                    0   
                    West                         0                    0   

                               cloud_cover_high (%)  wind_speed_10m (km/h)  \
datetime            region                                                   
2018-01-02 00:00:00 Bruce                         0                   32.2   
                    East                          0                   15.9   
                    Essa                          0                    5.9   
                    Niagara                       0                   11.7   
                    Northeast                   100                   11.5   
...                                             ...                    ...   
2018-01-06 03:00:00 Northwest                     0                   10.0   
                    Ottawa                       91                   18.8   
                    Southwest                     0                   10.7   
                    Toronto                       0                   10.5   
                    West                          0                   21.7   

                               wind_speed_100m (km/h)  \
datetime            region                              
2018-01-02 00:00:00 Bruce                        47.4   
                    East                         26.5   
                    Essa                         12.5   
                    Niagara                      22.7   
                    Northeast                    25.5   
...                                               ...   
2018-01-06 03:00:00 Northwest                    18.5   
                    Ottawa                       32.0   
                    Southwest                    24.9   
                    Toronto                      24.8   
                    West                         37.5   

                               weather_code (wmo code)  precipitation (mm)  \
datetime            region                                                   
2018-01-02 00:00:00 Bruce                           71                 0.2   
                    East                             1                 0.0   
                    Essa                             0        

In [148]:
display(climate_df)

temperature_2m (°C)  cloud_cover (%)  \
datetime            region                                            
2018-01-02 00:00:00 Bruce                     -6.8               98   
                    East                     -17.6               25   
                    Essa                     -22.6                4   
                    Niagara                  -12.4              100   
                    Northeast                -24.2              100   
...                                            ...              ...   
2025-04-04 23:00:00 Northwest                  0.1              100   
                    Ottawa                     1.9              100   
                    Southwest                  5.2              100   
                    Toronto                    5.0              100   
                    West                       5.4              100   

                               cloud_cover_low (%)  cloud_cover_mid (%)  \
datetime            region                                                
2018-01-02 00:00:00 Bruce                       96                   98   
                    East                        25                    0   
                    Essa                         4                    0   
                    Niagara                    100                  100   
                    Northeast                    0                  100   
...                                            ...                  ...   
2025-04-04 23:00:00 Northwest                   18                   99   
                    Ottawa                       0                    0   
                    Southwest                    0                   88   
                    Toronto                      0                  100   
                    West                         0                   57   

                               cloud_cover_high (%)  wind_speed_10m (km/h)  \
datetime            region                                                   
2018-01-02 00:00:00 Bruce                         0                   32.2   
                    East                          0                   15.9   
                    Essa                          0                    5.9   
                    Niagara                       0                   11.7   
                    Northeast                   100                   11.5   
...                                             ...                    ...   
2025-04-04 23:00:00 Northwest                    47                   12.4   
                    Ottawa                      100                    7.4   
                    Southwest                   100                   14.8   
                    Toronto                     100                    7.2   
                    West                        100                   19.1   

                               wind_speed_100m (km/h)  \
datetime            region                              
2018-01-02 00:00:00 Bruce                        47.4   
                    East                         26.5   
                    Essa                         12.5   
                    Niagara                      22.7   
                    Northeast                    25.5   
...                                               ...   
2025-04-04 23:00:00 Northwest                    24.6   
                    Ottawa                       15.0   
                    Southwest                    27.5   
                    Toronto                      17.6   
                    West                         35.8   

                               weather_code (wmo code)  precipitation (mm)  \
datetime            region                                                   
2018-01-02 00:00:00 Bruce                           71                 0.2   
                    East                             1                 0.0   
                    Essa                             0        

In [149]:
climate_df.head(11)

temperature_2m (°C)  cloud_cover (%)  \
datetime            region                                            
2018-01-02 00:00:00 Bruce                     -6.8               98   
                    East                     -17.6               25   
                    Essa                     -22.6                4   
                    Niagara                  -12.4              100   
                    Northeast                -24.2              100   
                    Northwest                -15.4               99   
                    Ottawa                   -22.6               92   
                    Southwest                -11.2               98   
                    Toronto                  -14.8               75   
                    West                     -14.1                8   
2018-01-02 01:00:00 Bruce                     -6.5               90   

                               cloud_cover_low (%)  cloud_cover_mid (%)  \
datetime            region                                                
2018-01-02 00:00:00 Bruce                       96                   98   
                    East                        25                    0   
                    Essa                         4                    0   
                    Niagara                    100                  100   
                    Northeast                    0                  100   
                    Northwest                    0                   99   
                    Ottawa                      92                    0   
                    Southwest                   95                   97   
                    Toronto                     75                    0   
                    West                         8                    0   
2018-01-02 01:00:00 Bruce                       85                   87   

                               cloud_cover_high (%)  wind_speed_10m (km/h)  \
datetime            region                                                   
2018-01-02 00:00:00 Bruce                         0                   32.2   
                    East                          0                   15.9   
                    Essa                          0                    5.9   
                    Niagara                       0                   11.7   
                    Northeast                   100                   11.5   
                    Northwest                    26                   23.1   
                    Ottawa                        0                    8.2   
                    Southwest                     0                   10.7   
                    Toronto                       0                    1.3   
                    West                          0                   19.3   
2018-01-02 01:00:00 Bruce                         0                   32.3   

                               wind_speed_100m (km/h)  \
datetime            region                              
2018-01-02 00:00:00 Bruce                        47.4   
                    East                         26.5   
                    Essa                         12.5   
                    Niagara                      22.7   
                    Northeast                    25.5   
                    Northwest                    40.3   
                    Ottawa                       18.2   
                    Southwest                    22.0   
                    Toronto                       5.6   
                    West                         33.9   
2018-01-02 01:00:00 Bruce                        47.1   

                               weather_code (wmo code)  precipitation (mm)  \
datetime            region                                                   
2018-01-02 00:00:00 Bruce                           71                 0.2   
                    East                             1                 0.0   
                    Essa                             0        

In [150]:
# combine demand and climate data

# melt demand data, i.e. make it such that each row is a unique datetime and region combination 
demand_df = demand_df.melt(
    id_vars=['datetime'],
    value_vars=['Northwest', 'Northeast', 'Ottawa', 'East', 'Toronto', 'Essa', 'Bruce', 'Southwest', 'Niagara', 'West'],
    var_name='region',
    value_name='zonal_demand'
)

# # merge 
merged_df = pd.merge(
    climate_df,
    demand_df[['datetime', 'region', 'zonal_demand']],
    on=['datetime', 'region'],
    how='left'  # Use 'inner' if you only want matching rows
)
merged_df.set_index(['datetime', 'region'], inplace=True)

display(merged_df)
# merged_df.shape



temperature_2m (°C)  cloud_cover (%)  \
datetime            region                                            
2018-01-02 00:00:00 Bruce                     -6.8               98   
                    East                     -17.6               25   
                    Essa                     -22.6                4   
                    Niagara                  -12.4              100   
                    Northeast                -24.2              100   
...                                            ...              ...   
2025-04-04 23:00:00 Northwest                  0.1              100   
                    Ottawa                     1.9              100   
                    Southwest                  5.2              100   
                    Toronto                    5.0              100   
                    West                       5.4              100   

                               cloud_cover_low (%)  cloud_cover_mid (%)  \
datetime            region                                                
2018-01-02 00:00:00 Bruce                       96                   98   
                    East                        25                    0   
                    Essa                         4                    0   
                    Niagara                    100                  100   
                    Northeast                    0                  100   
...                                            ...                  ...   
2025-04-04 23:00:00 Northwest                   18                   99   
                    Ottawa                       0                    0   
                    Southwest                    0                   88   
                    Toronto                      0                  100   
                    West                         0                   57   

                               cloud_cover_high (%)  wind_speed_10m (km/h)  \
datetime            region                                                   
2018-01-02 00:00:00 Bruce                         0                   32.2   
                    East                          0                   15.9   
                    Essa                          0                    5.9   
                    Niagara                       0                   11.7   
                    Northeast                   100                   11.5   
...                                             ...                    ...   
2025-04-04 23:00:00 Northwest                    47                   12.4   
                    Ottawa                      100                    7.4   
                    Southwest                   100                   14.8   
                    Toronto                     100                    7.2   
                    West                        100                   19.1   

                               wind_speed_100m (km/h)  \
datetime            region                              
2018-01-02 00:00:00 Bruce                        47.4   
                    East                         26.5   
                    Essa                         12.5   
                    Niagara                      22.7   
                    Northeast                    25.5   
...                                               ...   
2025-04-04 23:00:00 Northwest                    24.6   
                    Ottawa                       15.0   
                    Southwest                    27.5   
                    Toronto                      17.6   
                    West                         35.8   

                               weather_code (wmo code)  precipitation (mm)  \
datetime            region                                                   
2018-01-02 00:00:00 Bruce                           71                 0.2   
                    East                             1                 0.0   
                    Essa                             0        

In [154]:
merged_df.head(50)

temperature_2m (°C)  cloud_cover (%)  \
datetime            region                                            
2018-01-02 00:00:00 Bruce                     -6.8               98   
                    East                     -17.6               25   
                    Essa                     -22.6                4   
                    Niagara                  -12.4              100   
                    Northeast                -24.2              100   
                    Northwest                -15.4               99   
                    Ottawa                   -22.6               92   
                    Southwest                -11.2               98   
                    Toronto                  -14.8               75   
                    West                     -14.1                8   
2018-01-02 01:00:00 Bruce                     -6.5               90   
                    East                     -18.4                7   
                    Essa                     -22.4               79   
                    Niagara                  -12.4              100   
                    Northeast                -22.7              100   
                    Northwest                -15.3               97   
                    Ottawa                   -25.0               82   
                    Southwest                -11.1              100   
                    Toronto                  -17.1               45   
                    West                     -14.4                5   
2018-01-02 02:00:00 Bruce                     -6.2              100   
                    East                     -19.4                0   
                    Essa                     -21.9              100   
                    Niagara                  -12.3              100   
                    Northeast                -21.1              100   
                    Northwest                -15.2               95   
                    Ottawa                   -23.6               45   
                    Southwest                -11.0              100   
                    Toronto                  -16.4               99   
                    West                     -14.7                2   
2018-01-02 03:00:00 Bruce                     -6.3              100   
                    East                     -20.0                0   
                    Essa                     -21.1               96   
                    Niagara                  -13.1               90   
                    Northeast                -19.5              100   
                    Northwest                -15.3              100   
                    Ottawa                   -24.2                1   
                    Southwest                -10.7               98   
                    Toronto                  -17.7               98   
                    West                     -14.9                4   
2018-01-02 04:00:00 Bruce                     -6.4              100   
                    East                     -20.5                5   
                    Essa                     -21.1              100   
                    Niagara                  -13.2               30   
                    Northeast                -17.7              100   
                    Northwest                -15.4              100   
                    Ottawa                   -24.9               20   
                    Southwest                -10.8              100   
                    Toronto                  -14.3               99   
                    West                     -14.8                4   

                               cloud_cover_low (%)  cloud_cover_mid (%)  \
datetime            region                                                
2018-01-02 00:00:00 Bruce                       96                   98   
                    East                        25                    0   
                    Essa              

In [151]:
demand_df.tail(1000)

,datetime,region,zonal_demand
635240,2025-02-22 08:00:00,West,2169
635241,2025-02-22 09:00:00,West,2019
635242,2025-02-22 10:00:00,West,1783
635243,2025-02-22 11:00:00,West,1690
635244,2025-02-22 12:00:00,West,1641
...,...,...,...
636235,2025-04-04 19:00:00,West,1754
636236,2025-04-04 20:00:00,West,1698
636237,2025-04-04 21:00:00,West,1556
636238,2025-04-04 22:00:00,West,1447


In [152]:
def add_time_features(df):
    """Add time-based features to the dataframe."""
    df['hour'] = df['datetime'].dt.hour
    df['day_of_week'] = df['datetime'].dt.dayofweek
    df['month'] = df['datetime'].dt.month
    df['day_of_year'] = df['datetime'].dt.dayofyear
    
    # Add holiday information
    holiday_years = list(range(2018, 2026))
    ca_holidays = holidays.CA(prov='ON', years=holiday_years)
    print(ca_holidays)
    df['is_holiday'] = df['datetime'].dt.date.isin(ca_holidays)
    df['is_weekend'] = df['day_of_week'].isin([5, 6])
    
    return df

def create_lag_features(df, target_col, lags=[24, 48, 168]):
    """Create lagged features for the target variable."""
    for lag in lags:
        df[f'{target_col}_lag_{lag}'] = df[target_col].shift(lag)
    
    # Drop rows with NaN values in any of the lagged features
    # This ensures we only keep complete data points
    df = df.dropna(subset=[f'{target_col}_lag_{lag}' for lag in lags])
    
    return df

In [153]:
demand_df = add_time_features(demand_df)
# we can try adding time lag later, not sure how valuable it is 

# Save the combined data to a CSV file
display(demand_df)
output_file = 'zonal_demand.csv'
demand_df.to_csv(output_file, index=False)
print(f"Combined zonal demand data saved to {output_file}")


{datetime.date(2018, 1, 1): "New Year's Day", datetime.date(2018, 3, 30): 'Good Friday', datetime.date(2018, 7, 1): 'Canada Day', datetime.date(2018, 9, 3): 'Labour Day', datetime.date(2018, 12, 25): 'Christmas Day', datetime.date(2018, 2, 19): 'Family Day', datetime.date(2018, 5, 21): 'Victoria Day', datetime.date(2018, 10, 8): 'Thanksgiving Day', datetime.date(2018, 12, 26): 'Boxing Day', datetime.date(2019, 1, 1): "New Year's Day", datetime.date(2019, 4, 19): 'Good Friday', datetime.date(2019, 7, 1): 'Canada Day', datetime.date(2019, 9, 2): 'Labour Day', datetime.date(2019, 12, 25): 'Christmas Day', datetime.date(2019, 2, 18): 'Family Day', datetime.date(2019, 5, 20): 'Victoria Day', datetime.date(2019, 10, 14): 'Thanksgiving Day', datetime.date(2019, 12, 26): 'Boxing Day', datetime.date(2020, 1, 1): "New Year's Day", datetime.date(2020, 4, 10): 'Good Friday', datetime.date(2020, 7, 1): 'Canada Day', datetime.date(2020, 9, 7): 'Labour Day', datetime.date(2020, 12, 25): 'Christmas Da

,datetime,region,zonal_demand,hour,day_of_week,month,day_of_year,is_holiday,is_weekend
0,2018-01-01 00:00:00,Northwest,591,0,0,1,1,True,False
1,2018-01-01 01:00:00,Northwest,577,1,0,1,1,True,False
2,2018-01-01 02:00:00,Northwest,613,2,0,1,1,True,False
3,2018-01-01 03:00:00,Northwest,656,3,0,1,1,True,False
4,2018-01-01 04:00:00,Northwest,657,4,0,1,1,True,False
...,...,...,...,...,...,...,...,...,...
636235,2025-04-04 19:00:00,West,1754,19,4,4,94,False,False
636236,2025-04-04 20:00:00,West,1698,20,4,4,94,False,False
636237,2025-04-04 21:00:00,West,1556,21,4,4,94,False,False
636238,2025-04-04 22:00:00,West,1447,22,4,4,94,False,False


Combined zonal demand data saved to zonal_demand.csv
